In [1]:
import numpy as np
import pandas as pd
from stanfordcorenlp import StanfordCoreNLP
import time
import re
from datetime import datetime
nlp = StanfordCoreNLP(r'/home/cior5757e/ray/stanford-corenlp-full-2018-10-05', lang='zh')

In [2]:
def remove(list): 
    pattern = '[0-9]'
    list = [re.sub(pattern, '', i) for i in list] 
    return list

In [3]:
data = pd.read_csv('label_news.csv')[['title', 'content']]
data = np.array(data)

In [4]:
stock_words = []
with open('./stock.txt','r') as f:
    for i in f.readlines():
        i = i.strip()
        stock_words.append(i)

In [5]:
label = []
for obj in range(data.shape[0]):
    #label.append([])
    try:
        b = 0
        for i in stock_words:
            if data[obj][1].find(i) != -1:
                b = 1
                break
        if b:
            label.append(1)
        else:
            label.append(0)
    except:
        b = 0
        for i in stock_words:
            if data[obj][0].find(i) != -1:
                b = 1
                break
        if b:
            label.append(1)
        else:
            label.append(0)

label = np.array(label) # associate to stock or not

In [6]:
label = label.reshape(label.shape[0], 1)
data = np.hstack([data,label])
print(data.shape)
print(sum(data[:,2]))

(247273, 3)
138276


In [7]:
# delete not stocking data
del_row = []
for i in range(data.shape[0]):
    if data[i][2] == 0:
        del_row.append(i)
del_row = np.array(del_row)
np.save('del_row.npy', del_row)
data = np.delete(data, del_row, 0)
data = np.delete(data, 2, 1)
print(data.shape)

(138276, 2)


In [8]:
#del_row = np.load('delete_rows.npy', allow_pickle = True)
y = pd.read_csv('label_news.csv')[['label_3','label_5','label_10']]
y = np.array(y)
print(y.shape)
y = np.delete(y, del_row, axis=0)
print(y.shape)
del_row = []
for i in range(y.shape[0]):
    if y[i][0] == -1 and y[i][1] == -1 and y[i][2] == -1:
        del_row.append(i)

del_row = np.array(del_row)
np.save('del_nodata.npy', del_row)
y = np.delete(y, del_row, axis=0)
data = np.delete(data, del_row, axis=0)
print(data.shape)
print(y.shape)

(247273, 3)
(138276, 3)
(125058, 2)
(125058, 3)


In [9]:
for i in range(data.shape[0]):
    try:
        data[i] = remove(data[i])
    except:
        data[i][0] = re.sub('[0-9]', '', data[i][0])

In [10]:
mark = ["，","。","！","？","?","!",".","、","）","（","(",")"," ","「","」","『","』","【","】",",",
         '1','2','3','4','5','6','7','8','9','0'," ", "<BR>", "<", ">", ".", "：",":"]
nums = ['1','2','3','4','5','6','7','8','9','0']
stop_words = []
with open('./stop_words.txt','r') as f:
    for i in f.readlines():
        i = i.strip()
        stop_words.append(i)
stop_words += mark
stop_words = np.array(stop_words)

In [12]:
# 切詞會跑很久，這邊就不執行了
content = []
print(data.shape[0])
s = time.time()       
for i in range(data.shape[0]):
    if i % 10000 == 0:
        print(i, time.time()-s)
        s = time.time()
    try:
        segments = nlp.word_tokenize(data[i][1])
        tmp = list(filter(lambda a: a not in stop_words and a != '\n', segments))
        content.append(tmp)
    except:
        segments = nlp.word_tokenize(data[i][0])
        tmp = list(filter(lambda a: a not in stop_words and a != '\n', segments))
        content.append(tmp)
content = np.array(content)

125058
0 0.0003712177276611328
10000 355.3376576900482
20000 364.372211933136
30000 349.15840792655945
40000 337.12468457221985
50000 338.06488370895386
60000 291.32777404785156
70000 286.1272768974304
80000 293.25032329559326
90000 304.4039466381073
100000 298.9900288581848
110000 304.0956914424896
120000 307.8253107070923


In [13]:
month = np.load('month.npy', allow_pickle = True)
month = month.reshape(month.shape[0],1)
content = content.reshape(content.shape[0],1)
content = np.hstack([content,month])
content = np.hstack([content,y])
content.shape

(125058, 5)

In [15]:
np.save('train.npy',content)